## BDD

In [ ]:
pip install PyPDF2 pandas scikit-learn

In [ ]:
import os
from PyPDF2 import PdfReader
import csv
import pandas as pd
from sklearn.utils import resample

# Fonction pour extraire le texte d'un PDF
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading {pdf_path}: {str(e)}"

# Dossier contenant les PDF
directory = "BDD"
data = []

# Vérifier si le dossier existe
if not os.path.exists(directory):
    print(f"Le dossier '{directory}' n'existe pas.")
else:
    # Parcourir tous les fichiers dans le dossier
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            #print(f"Extraction du contenu de : {filename}")
            extracted_text = extract_text_from_pdf(pdf_path)
            data.append([filename, extracted_text])

    # Créer un DataFrame pour manipuler les données
    pdf_dataframe = pd.DataFrame(data, columns=["Title", "Content"])

    # Nettoyer les titres
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'\d+', '', regex=True)  # Supprimer les nombres
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'\.pdf', '', regex=True)  # Supprimer ".pdf"
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'[_-]', '', regex=True)  # Supprimer "_" et "-"

    # Remplacer les valeurs spécifiques dans la colonne Title
    pdf_dataframe['Title'] = pdf_dataframe['Title'].replace({
        'assignation': 0,
        'assignationsansVices': 1,
        'NotificationVice': 2,
        'Notification': 3
    })

    # Afficher la répartition des classes avant équilibrage
    print("Répartition des classes avant équilibrage :")
    print(pdf_dataframe['Title'].value_counts())

    # Séparer les classes majoritaires et minoritaires
    majority = pdf_dataframe[pdf_dataframe['Title'] == 0]
    minority_1 = pdf_dataframe[pdf_dataframe['Title'] == 1]
    minority_2 = pdf_dataframe[pdf_dataframe['Title'] == 2]
    minority_3 = pdf_dataframe[pdf_dataframe['Title'] == 3]

    # Suréchantillonner les classes minoritaires
    minority_1_upsampled = resample(minority_1, 
                                    replace=True,     # Permet le suréchantillonnage
                                    n_samples=len(majority),  # Faire correspondre la taille de la classe majoritaire
                                    random_state=123)
    minority_2_upsampled = resample(minority_2, 
                                    replace=True, 
                                    n_samples=len(majority), 
                                    random_state=123)
    minority_3_upsampled = resample(minority_3, 
                                    replace=True, 
                                    n_samples=len(majority), 
                                    random_state=123)

    # Fusionner toutes les classes après le suréchantillonnage
    data_balanced = pd.concat([majority, minority_1_upsampled, minority_2_upsampled, minority_3_upsampled])

    # Sauvegarder le contenu équilibré dans un fichier CSV
    output_file = "pdf_contents_final.csv"
    data_balanced.to_csv(output_file, index=False, encoding="utf-8")

    print(f"Le contenu équilibré a été sauvegardé dans '{output_file}'.")

    # Afficher la répartition des classes après équilibrage
    print("Répartition des classes après équilibrage :")
    print(data_balanced['Title'].value_counts())

    # Afficher les premières lignes du DataFrame équilibré
    #print("Aperçu des données équilibrées :")
    #print(data_balanced.head())


## Legal BERT V2

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Charger le dataset
df = pd.read_csv('pdf_contents_final.csv')

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['Title'], test_size=0.2, random_state=42)

# Charger le tokenizer de LegalBERT
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

# Tokenisation des données
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512)

train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())

# Créer un Dataset personnalisé pour BERT
class LegalBERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

# Créer les datasets pour l'entraînement et l'évaluation
train_dataset = LegalBERTDataset(train_encodings, y_train)
test_dataset = LegalBERTDataset(test_encodings, y_test)

# Charger le modèle pré-entraîné de LegalBERT pour la classification
model = BertForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=4)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    evaluation_strategy="epoch",     # Stratégie d'évaluation par époque
    learning_rate=2e-5,              # Taux d'apprentissage
    per_device_train_batch_size=8,   # Taille du batch pour l'entraînement
    per_device_eval_batch_size=8,    # Taille du batch pour l'évaluation
    num_train_epochs=3,              # Nombre d'époques
    weight_decay=0.01,               # Décroissance du poids
    logging_dir='./logs',            # Répertoire des logs
)

# Initialiser le Trainer
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()

# Prédictions sur le jeu de test
predictions = trainer.predict(test_dataset)

# Afficher les résultats des prédictions
print(predictions.predictions.argmax(axis=-1))

# Sauvegarder le modèle et le tokenizer
model.save_pretrained('./legal_bert_model')
tokenizer.save_pretrained('./legal_bert_model')


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6170464754104614, 'eval_runtime': 5.4151, 'eval_samples_per_second': 13.665, 'eval_steps_per_second': 1.847, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.4249761998653412, 'eval_runtime': 5.6577, 'eval_samples_per_second': 13.08, 'eval_steps_per_second': 1.768, 'epoch': 2.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.3963528573513031, 'eval_runtime': 5.7534, 'eval_samples_per_second': 12.862, 'eval_steps_per_second': 1.738, 'epoch': 3.0}
{'train_runtime': 217.9124, 'train_samples_per_second': 4.047, 'train_steps_per_second': 0.509, 'train_loss': 0.6449106577280406, 'epoch': 3.0}


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

[1 0 1 3 1 2 1 1 1 1 2 0 1 0 0 2 0 2 3 0 0 3 0 2 2 2 2 2 3 3 0 1 0 3 1 1 0
 1 3 1 2 2 0 2 1 3 0 0 1 0 2 3 3 3 1 3 1 0 2 1 2 2 0 0 3 1 2 1 2 0 1 2 3 3]


('./legal_bert_model/tokenizer_config.json',
 './legal_bert_model/special_tokens_map.json',
 './legal_bert_model/vocab.txt',
 './legal_bert_model/added_tokens.json')

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, confusion_matrix

# Charger le dataset
df = pd.read_csv('pdf_contents_final.csv')

# Charger le modèle et le tokenizer sauvegardés
model = BertForSequenceClassification.from_pretrained('./legal_bert_model')
tokenizer = BertTokenizer.from_pretrained('./legal_bert_model')

# Fonction pour prédire la classe d'un texte
def predict(texts):
    # Tokeniser les nouveaux textes
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    
    # Obtenir les prédictions du modèle
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
    
    # Obtenir les indices des classes prédominantes
    predictions = torch.argmax(logits, dim=-1)
    return predictions

# Tester le modèle avec le contenu du dataset
new_texts = df['Content'].tolist()  # Liste des contenus des PDFs
predicted_labels = predict(new_texts)

# Ajouter les prédictions dans le dataframe
df['Predicted_Label'] = predicted_labels.numpy()

# Afficher les résultats avec les titres
print("Exemples de prédictions comparées aux véritables titres :")
print(df[['Title', 'Predicted_Label']].head())

# Calcul de la précision
accuracy = accuracy_score(df['Title'], df['Predicted_Label'])
print(f"Précision du modèle : {accuracy * 100:.2f}%")

# Confusion Matrix pour visualiser les bonnes et mauvaises prédictions
conf_matrix = confusion_matrix(df['Title'], df['Predicted_Label'])
print("\nMatrice de confusion :")
print(conf_matrix)

# Nombre de bonnes réponses
correct_predictions = (df['Title'] == df['Predicted_Label']).sum()
print(f"\nNombre de bonnes réponses : {correct_predictions}")

# Nombre de mauvaises réponses
incorrect_predictions = len(df) - correct_predictions
print(f"Nombre de mauvaises réponses : {incorrect_predictions}")


Exemples de prédictions comparées aux véritables titres :
   Title  Predicted_Label
0      0                0
1      0                0
2      0                0
3      0                0
4      0                0
Précision du modèle : 89.40%

Matrice de confusion :
[[63 29  0  0]
 [10 82  0  0]
 [ 0  0 92  0]
 [ 0  0  0 92]]

Nombre de bonnes réponses : 329
Nombre de mauvaises réponses : 39
